In [1]:
!git clone https://github.com/Tselmuun0624/Neural_PDE-Final.git

Cloning into 'Neural_PDE-Final'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 102 (delta 41), reused 91 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (102/102), 1.16 MiB | 3.08 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
import os
os.chdir('./Neural_PDE-Final')
print(os.getcwd())

/content/Neural_PDE-Final


In [3]:
!ls

experiments  README.md	torchsde


In [4]:
!pip install torchsde

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.3 MB/s eta 0:00:00


In [5]:
########### MNIST Dataset Traning Experiment ##############

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
from torchvision import datasets, transforms
import torchsde
from tqdm import tqdm
import numpy as np

class MNISTSDEModel(torchsde.SDEIto):
    def __init__(self, data_dim, hidden_dim=64, noise_type='diagonal'):
        super().__init__(noise_type=noise_type)

        self.fc1 = nn.Linear(data_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, data_dim)
        self.sigma = nn.Parameter(torch.ones(data_dim))

    def f(self, t, x):
        # Drift function
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        return self.fc3(h)

    def g(self, t, x):
        # Diffusion function
        return self.sigma * torch.ones_like(x)

    def h(self, t, x):
        # Aux function for SDE
        return torch.zeros_like(x)

    def sample(self, batch_size, x0, t, dt):

        num_steps = int(t / dt)
        xs = [x0]
        for _ in range(num_steps):
            t = torch.tensor([0.0, dt])
            x = torchsde.sdeint(self, xs[-1], t, dt=dt)[-1]
            xs.append(x)
        return torch.stack(xs)

def train_mnist_sde(epochs=10, batch_size=128, learning_rate=0.001, validation_split=0.5):
    # Normalizing data for our model
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load MNIST Dataset
    full_dataset = datasets.MNIST(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )

    # Split the training and test dataset equally
    train_size = int(len(full_dataset) * (1 - validation_split))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    input_dim = 28 * 28
    model = MNISTSDEModel(data_dim=input_dim)

    classifier = nn.Sequential(
        nn.Linear(input_dim, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    )

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(list(model.parameters()) + list(classifier.parameters()), lr=learning_rate)

    train_losses = []
    test_accuracies = []

    for epoch in range(epochs):

        model.train()
        total_train_loss = 0.0

        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}',
                          postfix={'train_loss': 0.0, 'test_accuracy': 0.0})

        for batch_idx, (data, target) in enumerate(train_pbar):

            data = data.view(data.size(0), -1)

            batch_size = data.size(0)
            t0, t1 = 0.0, 1.0
            ts = torch.tensor([t0, t1])
            xs = torchsde.sdeint(model, data, ts, dt=0.1)

            outputs = classifier(xs[-1])
            loss = criterion(outputs, target)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            train_pbar.set_postfix({
                'train_loss': f'{loss.item():.4f}'
            })

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            test_pbar = tqdm(test_loader, desc='Testing', leave=False)
            for data, target in test_pbar:
                data = data.view(data.size(0), -1)
                xs = model.sample(data.size(0), x0=data, t=t0, dt=0.1)
                outputs = classifier(xs[-1])

                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

                test_pbar.set_postfix({'accuracy': f'{100 * correct / total:.2f}%'})

        test_accuracy = 100 * correct / total
        test_accuracies.append(test_accuracy)

        train_pbar.close()

        print(f'Epoch {epoch+1}:')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Test Accuracy: {test_accuracy:.2f}%')

    print(f"Best Test Accuracy: {max(test_accuracies):.2f}%")

    return model, classifier, train_losses, test_accuracies

if __name__ == '__main__':

    torch.manual_seed(42)
    np.random.seed(42)

    model, classifier, train_losses, test_accuracies = train_mnist_sde(epochs=10)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.9MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 493kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Epoch 8:
  Train Loss: 0.0537
  Test Accuracy: 69.99%


Epoch 9/10: 100%|██████████| 235/235 [01:34<00:00,  2.48it/s, train_loss=0.0542]


Epoch 9:
  Train Loss: 0.0498
  Test Accuracy: 64.37%


Epoch 10/10: 100%|██████████| 235/235 [01:34<00:00,  2.50it/s, train_loss=0.0847]
                                                                           

Epoch 10:
  Train Loss: 0.0433
  Test Accuracy: 63.69%
Best Test Accuracy: 79.71%
